
# SparkR tutorial Notebook
This is a notebook version of the [SparkR Documentation](http://spark.apache.org/docs/2.4.0/sparkr.html)
## Overview
SparkR is an R package that provides a light-weight frontend to use Apache Spark from R. In Spark 2.4.0, SparkR provides a distributed data frame implementation that supports operations like selection, filtering, aggregation etc. (similar to R data frames, dplyr) but on large datasets. SparkR also supports distributed machine learning using MLlib.

# SparkDataFrame
A SparkDataFrame is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R, but with richer optimizations under the hood. SparkDataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing local R data frames.

All of the examples on this page use sample data included in R or the Spark distribution and can be run using the ./bin/sparkR shell.

## Starting Up: SparkSession
You can start SparkR from Jupyter R Kernel. You can connect your R program to a Spark cluster from RStudio, R shell, Rscript or other R IDEs. To start, make sure SPARK_HOME is set in environment (you can check Sys.getenv), load the SparkR package, and call sparkR.session as below. It will check for the Spark installation, and, if not found, it will be downloaded and cached automatically. Alternatively, you can also run install.spark manually.

In addition to calling sparkR.session, you could also specify certain Spark driver properties. Normally these Application properties and Runtime Environment cannot be set programmatically, as the driver JVM process would have been started, in this case SparkR takes care of this for you. To set them, pass them as you would other configuration properties in the sparkConfig argument to sparkR.session().


## Spark Master

You can configure your spark master url when starting up SparkSession.

To use PNDA spark cluster, get the master url from the environment variable SPARKCONF_SPARK_MASTER


In [ ]:
spark_master = Sys.getenv("SPARKCONF_SPARK_MASTER")

In [ ]:
if (nchar(Sys.getenv("SPARK_HOME")) < 1) {
  Sys.setenv(SPARK_HOME = "/home/spark")
}
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))
sparkR.session(master = spark_master, sparkConfig = list(spark.driver.memory = "2g"))

The following Spark driver properties can be set in sparkConfig with sparkR.session from RStudio:

Property Name | Property group | spark-submit equivalent
 --- | --- | --- 
spark.master |	Application Properties	| --master
spark.yarn.keytab|Application Properties|	--keytab
spark.yarn.principal|	Application Properties|	--principal
spark.driver.memory|	Application Properties|	--driver-memory
spark.driver.extraClassPath|	Runtime Environment|	--driver-class-path
spark.driver.extraJavaOptions|	Runtime Environment|	--driver-java-options
spark.driver.extraLibraryPath|	Runtime Environment|	--driver-library-path

## Creating SparkDataFrames
With a SparkSession, applications can create SparkDataFrames from a local R data frame, from a Hive table, or from other data sources.

### From local data frames
The simplest way to create a data frame is to convert a local R data frame into a SparkDataFrame. Specifically, we can use as.DataFrame or createDataFrame and pass in the local R data frame to create a SparkDataFrame. As an example, the following creates a SparkDataFrame based using the faithful dataset from R.

In [ ]:
df <- as.DataFrame(faithful)
# Displays the first part of the SparkDataFrame
head(df)

### From Data Sources
SparkR supports operating on a variety of data sources through the SparkDataFrame interface. This section describes the general methods for loading and saving data using Data Sources. You can check the Spark SQL programming guide for more specific options that are available for the built-in data sources.

The general method for creating SparkDataFrames from data sources is read.df. This method takes in the path for the file to load and the type of data source, and the currently active SparkSession will be used automatically. SparkR supports reading JSON, CSV and Parquet files natively, and through packages available from sources like Third Party Projects, you can find data source connectors for popular file formats like Avro. These packages can either be added by specifying --packages with spark-submit or sparkR commands, or if initializing SparkSession with sparkPackages parameter when in an interactive R shell or from RStudio.

In [ ]:
sparkR.session(sparkPackages = "com.databricks:spark-avro_2.11:3.0.0")

We can see how to use data sources using an example JSON input file. Note that the file that is used here is not a typical JSON file. Each line in the file must contain a separate, self-contained valid JSON object. For more information, please see JSON Lines text format, also called newline-delimited JSON. As a consequence, a regular multi-line JSON file will most often fail.

In [ ]:
people <- read.df("/opt/spark/examples/src/main/resources/people.json", "json")
head(people)

SparkR automatically infers the schema from the JSON file

In [ ]:
printSchema(people)

Similarly, multiple files can be read with read.json

In [ ]:
people <- read.json(c("/opt/spark/examples/src/main/resources/people.json", "/opt/spark/examples/src/main/resources/people.json"))

The data sources API natively supports CSV formatted input files. For more information please refer to SparkR [read.df](http://spark.apache.org/docs/latest/api/R/read.df.html) API documentation.

In [ ]:
csvPath <- "/opt/spark/examples/src/main/resources/people.csv"
df <- read.df(csvPath, "csv", header = "true", inferSchema = "true", na.strings = "NA", delimiter=";")

In [ ]:
head(df)

### From Hive tables (NOT DONE)
You can also create SparkDataFrames from Hive tables. To do this we will need to create a SparkSession with Hive support which can access tables in the Hive MetaStore. Note that Spark should have been built with Hive support and more details can be found in the SQL programming guide. In SparkR, by default it will attempt to create a SparkSession with Hive support enabled (enableHiveSupport = TRUE).

# SparkDataFrame Operations
SparkDataFrames support a number of functions to do structured data processing. Here we include some basic examples and a complete list can be found in the API docs:

## Selecting rows, columns

In [ ]:
df <- as.DataFrame(faithful)
# Get basic information about the SparkDataFrame
df

In [ ]:
# Select only the "eruptions" column
head(select(df, df$eruptions))

In [ ]:
# You can also pass in column name as strings
head(select(df, "eruptions"))

In [ ]:
# Filter the SparkDataFrame to only retain rows with wait times shorter than 50 mins
head(filter(df, df$waiting < 50))
##  eruptions waiting
##1     1.750      47
##2     1.750      47
##3     1.867      48

## Grouping, Aggregation
SparkR data frames support a number of commonly used functions to aggregate data after grouping. For example, we can compute a histogram of the waiting time in the faithful dataset as shown below

In [ ]:
# We use the `n` operator to count the number of times each waiting time appears
head(summarize(groupBy(df, df$waiting), count = n(df$waiting)))

In [ ]:
# We can also sort the output from the aggregation to get the most common waiting times
waiting_counts <- summarize(groupBy(df, df$waiting), count = n(df$waiting))
head(arrange(waiting_counts, desc(waiting_counts$count)))

In addition to standard aggregations, SparkR supports OLAP cube operators cube:

In [ ]:
df <- as.DataFrame(mtcars)
head(agg(cube(df, "cyl", "disp", "gear"), avg(df$mpg)))

and rollup:

In [ ]:
head(agg(rollup(df, "cyl", "disp", "gear"), avg(df$mpg)))

## Operating on Columns
SparkR also provides a number of functions that can directly applied to columns for data processing and during aggregation. The example below shows the use of basic arithmetic functions.

In [ ]:
df <- as.DataFrame(faithful)
# Convert waiting time from hours to seconds.
# Note that we can assign this to a new column in the same SparkDataFrame
df$waiting_secs <- df$waiting * 60
head(df)

## Applying User-Defined Function
In SparkR, we support several kinds of User-Defined Functions:

### Run a given function on a large dataset using dapply or dapplyCollect
#### dapply
Apply a function to each partition of a SparkDataFrame. The function to be applied to each partition of the SparkDataFrame and should have only one parameter, to which a data.frame corresponds to each partition will be passed. The output of function should be a data.frame. Schema specifies the row format of the resulting a SparkDataFrame. It must match to data types of returned value.

In [ ]:
df <- as.DataFrame(faithful)

# Convert waiting time from hours to seconds.
# Note that we can apply UDF to DataFrame.
schema <- structType(structField("eruptions", "double"), structField("waiting", "double"), structField("waiting_secs", "double"))
head(collect(df))
df1 <- dapply(df, function(x) { x <- cbind(x, x$waiting * 60) }, schema)
head(collect(df1))

#### dapplyCollect
Like dapply, apply a function to each partition of a SparkDataFrame and collect the result back. The output of function should be a data.frame. But, Schema is not required to be passed. Note that dapplyCollect can fail if the output of UDF run on all the partition cannot be pulled to the driver and fit in driver memory.



In [ ]:
# Convert waiting time from hours to seconds.
# Note that we can apply UDF to DataFrame and return a R's data.frame
ldf <- dapplyCollect(
         df,
         function(x) {
           x <- cbind(x, "waiting_secs" = x$waiting * 60)
         })
head(ldf, 3)

### Run a given function on a large dataset grouping by input column(s) and using gapply or gapplyCollect
#### gapply
Apply a function to each group of a SparkDataFrame. The function is to be applied to each group of the SparkDataFrame and should have only two parameters: grouping key and R data.frame corresponding to that key. The groups are chosen from SparkDataFrames column(s). The output of function should be a data.frame. Schema specifies the row format of the resulting SparkDataFrame. It must represent R function's output schema on the basis of Spark data types. The column names of the returned data.frame are set by user.

In [ ]:
df <- as.DataFrame(faithful)

# Determine six waiting times with the largest eruption time in minutes.
schema <- structType(structField("waiting", "double"), structField("max_eruption", "double"))
result <- gapply(
    df,
    "waiting",
    function(key, x) {
        y <- data.frame(key, max(x$eruptions))
    },
    schema)
head(collect(arrange(result, "max_eruption", decreasing = TRUE)))


#### gapplyCollect
Like gapply, applies a function to each partition of a SparkDataFrame and collect the result back to R data.frame. The output of the function should be a data.frame. But, the schema is not required to be passed. Note that gapplyCollect can fail if the output of UDF run on all the partition cannot be pulled to the driver and fit in driver memory.

In [ ]:
# Determine six waiting times with the largest eruption time in minutes.
result <- gapplyCollect(
    df,
    "waiting",
    function(key, x) {
        y <- data.frame(key, max(x$eruptions))
        colnames(y) <- c("waiting", "max_eruption")
        y
    })
head(result[order(result$max_eruption, decreasing = TRUE), ])

### Run local R functions distributed using spark.lapply
#### spark.lapply
Similar to lapply in native R, spark.lapply runs a function over a list of elements and distributes the computations with Spark. Applies a function in a manner that is similar to doParallel or lapply to elements of a list. The results of all the computations should fit in a single machine. If that is not the case they can do something like df <- createDataFrame(list) and then use dapply



In [ ]:
# Perform distributed training of multiple models with spark.lapply. Here, we pass
# a read-only list of arguments which specifies family the generalized linear model should be.
families <- c("gaussian", "poisson")
train <- function(family) {
  model <- glm(Sepal.Length ~ Sepal.Width + Species, iris, family = family)
  summary(model)
}
# Return a list of model's summaries
model.summaries <- spark.lapply(families, train)

# Print the summary of each model
print(model.summaries)

## Running SQL Queries from SparkR
A SparkDataFrame can also be registered as a temporary view in Spark SQL and that allows you to run SQL queries over its data. The sql function enables applications to run SQL queries programmatically and returns the result as a SparkDataFrame.

In [ ]:
# Load a JSON file
people <- read.df("/opt/spark/examples/src/main/resources/people.json", "json")

# Register this SparkDataFrame as a temporary view.
createOrReplaceTempView(people, "people")

# SQL statements can be run by using the sql method
teenagers <- sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")
head(teenagers)

## Machine Learning

### Algorithms
SparkR supports the following machine learning algorithms currently:

#### Classification
- spark.logit: Logistic Regression
- spark.mlp: Multilayer Perceptron (MLP)
- spark.naiveBayes: Naive Bayes
- spark.svmLinear: Linear Support Vector Machine

#### Regression
- spark.survreg: Accelerated Failure Time (AFT) Survival Model
- spark.glm or glm: Generalized Linear Model (GLM)
- spark.isoreg: Isotonic Regression

#### Tree
- spark.decisionTree: Decision Tree for Regression and Classification
- spark.gbt: Gradient Boosted Trees for Regression and Classification
- spark.randomForest: Random Forest for Regression and Classification

#### Clustering
- spark.bisectingKmeans: Bisecting k-means
- spark.gaussianMixture: Gaussian Mixture Model (GMM)
- spark.kmeans: K-Means
- spark.lda: Latent Dirichlet Allocation (LDA)

#### Collaborative Filtering
- spark.als: Alternating Least Squares (ALS)
- Frequent Pattern Mining
- spark.fpGrowth : FP-growth

#### Statistics
- spark.kstest: Kolmogorov-Smirnov Test

Under the hood, SparkR uses MLlib to train the model. Please refer to the corresponding section of MLlib user guide for example code. Users can call summary to print a summary of the fitted model, predict to make predictions on new data, and write.ml/read.ml to save/load fitted models. SparkR supports a subset of the available R formula operators for model fitting, including '~', '.', ':', '+', and '-'.

### Model persistence
The following example shows how to save/load a MLlib model by SparkR.



In [ ]:
training <- read.df("/opt/spark/data/mllib/sample_multiclass_classification_data.txt", source = "libsvm")
# Fit a generalized linear model of family "gaussian" with spark.glm
df_list <- randomSplit(training, c(7,3), 2)
gaussianDF <- df_list[[1]]
gaussianTestDF <- df_list[[2]]
gaussianGLM <- spark.glm(gaussianDF, label ~ features, family = "gaussian")

# Save and then load a fitted MLlib model
modelPath <- tempfile(pattern = "ml", fileext = ".tmp")
write.ml(gaussianGLM, modelPath)
gaussianGLM2 <- read.ml(modelPath)

# Check model summary
summary(gaussianGLM2)

# Check model prediction
gaussianPredictions <- predict(gaussianGLM2, gaussianTestDF)
head(gaussianPredictions)

unlink(modelPath)

Find full example code at "examples/src/main/r/ml/ml.R" in the Spark repo.

In [ ]:
sparkR.session.stop()

## Data type mapping between R and Spark

<table class="table">
  <thead>
    <tr>
      <th>R</th>
      <th>Spark</th>
    </tr>
  </thead>
  <tbody>
    <tr>
        <td>byte</td><td>byte</td>
    </tr>
    <tr>
        <td>integer</td><td>integer</td>
    </tr>
    <tr>
        <td>float</td><td>float</td>
    </tr>
    <tr>
        <td>double</td><td>double</td>
    </tr>
    <tr>
        <td>numeric</td><td>double</td>
    </tr>
    <tr>
        <td>character</td><td>string</td>
    </tr>
    <tr>
        <td>string</td><td>string</td>
    </tr>
    <tr>
        <td>binary</td><td>binary</td>
    </tr>
    <tr>
        <td>raw</td><td></td>
    </tr>
    <tr>
        <td>logical</td><td>boolean</td>
    </tr>
    <tr>
        <td>POSIXct</td><td>timestamp</td>
    </tr>
    <tr>
        <td>POSIXlt</td><td>timestamp</td>
    </tr>
    <tr>
        <td>Date</td><td>date</td>
    </tr>
    <tr>
        <td>array</td><td>array</td>
    </tr>
    <tr>
        <td>list</td><td>array</td>
    </tr>
    <tr>
        <td>env</td><td>map</td>
    </tr>
    </tbody>
     <tfoot><tr><td colspan="4"></td></tr></tfoot>
</table>
